In [21]:
import numpy as np
import plotly.graph_objects as go

In [22]:
class SquaredExponentialKernel:
    def __init__(self, sigma_f: float = 1, length: float = 1):
        self.sigma_f = sigma_f 
        self.length = length
    
    def __call__(self, argument_1:np.array, argument_2: np.array) -> float:
        return float(self.sigma_f *
                     np.exp(-(np.linalg.norm(argument_1 - argument_2) ** 2) / 
                        (2* self.length**2)))

In [23]:
print(np.finfo(float).eps)
print(np.finfo(np.float32).eps)

2.220446049250313e-16
1.1920929e-07


In [24]:
def cov_matrix(x1, x2, cov_function) -> np.array:
    return np.array([[cov_function(a,b) for a in x1] for b in x2])

$$\mathcal{N}(K_*K^{-1}y,K_{**}-K_*K^{-1}K_*^T)$$
with 
$$K= (k(x_i,x_j))_{i,j\leq n}+\sigma^2*1_n$$
$$K_*= (k(x_i,x_j))_{n+1\leq i, j\leq n}$$
$$K_{**}= (k(x_i,x_j))_{n+1\leq i,j}$$

In [25]:
class GPR:
    def __init__(self,
                 data_x: np.array,
                 data_y: np.array,
                 covariance_function = SquaredExponentialKernel(),
                 white_noise_sigma: float = 0):
        self.noise = white_noise_sigma
        self.data_x = data_x
        self.data_y = data_y
        self.covariance_function = covariance_function

        self._inverse_of_covariance_matrix_of_input = np.linalg.inv(
            cov_matrix(data_x, data_x, covariance_function) + 
            (3e-7+self.noise) * np.identity(len(self.data_x)))
        self._memory = None
    def predict(self, at_values: np.array) -> np.array:
        k_lower_left = cov_matrix(self.data_x, at_values, self.covariance_function)
        k_lower_right =  cov_matrix(at_values, at_values, self.covariance_function)

        # Mean
        mean_at_values = np.dot(
            k_lower_left,
            np.dot(self.data_y, 
                   self._inverse_of_covariance_matrix_of_input.T).T).flatten()
        
        #Covariance
        cov_at_values = k_lower_right - \
            np.dot(k_lower_left, np.dot(
                self._inverse_of_covariance_matrix_of_input, k_lower_left.T
            ))
        
        cov_at_values = cov_at_values + 3e7 * np.ones(np.shape(cov_at_values)[0])

        var_at_values = np.diag(cov_at_values)

        self._memory = {
            'mean': mean_at_values,
            'covariance_matrix': cov_at_values,
            'variance': var_at_values
        }
        return mean_at_values

In [26]:
def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted function',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="blue", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )

In [27]:
def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points',
    showlegend: bool = True
    ) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=8),
        showlegend=showlegend
    )

In [28]:
def uncertainty_area_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_upper: np.array = np.array([]),
    y_lower: np.array = np.array([]),
    name: str = "mean plus/minus standard deviation",
    ) -> go.Scatter:

    return go.Scatter(
    visible=visible,
    x=np.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
    # upper, then lower reversed
    y=np.concatenate((y_upper, y_lower[::-1])),
    fill='toself',
    fillcolor='rgba(189,195,199,0.5)',
    line=dict(color='rgba(200,200,200,0)'),
    hoverinfo="skip",
    showlegend=True,
    name= name,
    )

In [29]:
def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    mean = model.predict(x)

    std = np.sqrt(model._memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(
            uncertainty_area_scatter(
                x_lines=x,
                y_lower=mean - i * std,
                y_upper=mean + i * std,
                name=f"mean plus/minus {i}*standard deviation",
                visible=visible))

    data.append(line_scatter(x_lines=x, y_lines=mean, visible=visible))
    data.append(dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible))
    return data

In [30]:
def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=600,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
        
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return fig

**Testing the function**


In [31]:
x_values = np.array([0, 0.3, 1, 3.1, 4.7])

y_values = np.array([1, 0, 1.4, 0, -0.9])

In [32]:
x = np.arange(-1,7,0.1)

In [35]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model)
fig4 = go.Figure(data=data)
fig4 = update_layout_of_graph(fig=fig4,
                              title='GPR with length 1, sigma 0 and noise 0')

fig4.show()